In [2]:
from arc_prize.train import ARCModelState, ARCTrainParams
from arc_prize.vis import visualize_epochs
import modal
import torch
import petname
from arc_prize.model import ARCTransformerEncoderDecoderParams



In [ ]:
from arc_prize.data import ARCDatasetParams, ReARCDataset, make_re_arc_data_loaders

config = ARCDatasetParams(max_grid_size=30, max_train_grids=10, color_offset=1)
dataset = ReARCDataset("data/re_arc/ff805c23.json", config)
train_loader, val_loader = make_re_arc_data_loaders(["data/re_arc/ff805c23.json"], 10, config)
len(val_loader.dataset)


In [ ]:
import modal
from arc_prize.train import ARCTrainParams
from arc_prize.model import ARCTransformerEncoderDecoderParams
import petname

model_type = "encoder_decoder"


model_params = ARCTransformerEncoderDecoderParams(
  grid_dim=12,
  num_train_pairs=4,
  num_colors=10,
  num_encoder_layers=16,
  num_decoder_layers=16,
  num_heads=16,
  d_model=512,
  d_ff=512*6,
  dropout=0.2
)

train_params = ARCTrainParams(
  batch_size=12,
  learning_rate=1e-4,
  weight_decay=1e-4,
  dataset_dir=["/vol/data/chunked/html_dim_12_20241108", "/vol/data/chunked/re_arc_dim_12", "/vol/data/chunked/barc_1_dim_12", "/vol/data/chunked/barc_2_dim_12"],
  loss_class_weights={0: 0.2},
  train_steps_per_epoch=1000,
  eval_steps_per_epoch=150,
  warmup_epochs=8,
  refinement_ratio=0.25
)

num_epochs = 300

model_names = []

num_runs = 1

fn = modal.Function.lookup("arc-prize", "train")
for i in range(num_runs):
  model_name = petname.generate(words=3, separator='_')
  fn_call = fn.spawn(model_name, num_epochs, model_type, model_params, train_params)
  print("Model name", model_name, model_type, fn_call.object_id)
  model_names.append(model_name)

print(model_names)




In [ ]:


from arc_prize.train import ARCTrainParams, train_on_mac
import petname
from arc_prize.model import ARCTransformerEncoderDecoderParams


model_type = "vision_encoder"
model_params = ARCTransformerEncoderDecoderParams(
  grid_dim=20,
  num_train_pairs=4,
  num_colors=10,
  num_encoder_layers=2,
  num_decoder_layers=0,
  num_heads=2,
  d_model=16,
  d_ff=16*2,
  dropout=0.1
)

train_params = ARCTrainParams(
  batch_size=12,
  learning_rate=1e-4,
  weight_decay=1e-4,
  dataset_dir=["/Users/pfh/work/arc-data/chunked/html_dim_12_20241023"],
  train_steps_per_epoch=3,
  eval_steps_per_epoch=2,
)

num_epochs = 4

model_name = petname.generate(words=3, separator='_')  
print(model_name)
train_on_mac(model_name, num_epochs, model_type, model_params, train_params)





In [ ]:
model_type = "vision"
model_params = ARCTransformerEncoderDecoderParams(
  grid_dim=12,
  num_train_pairs=4,
  num_colors=10,
  num_encoder_layers=1,
  num_decoder_layers=1,
  num_heads=4,
  d_model=32,
  d_ff=32*4,
  dropout=0.2
)

train_params = ARCTrainParams(
  batch_size=32,
  learning_rate=1e-4,
  weight_decay=1e-4,
  dataset_dir=["/vol/data/move_random_small", "/vol/data/flip"],
  loss_class_weights={0: 0.2},
  meta_num_epochs=2,
  meta_batch_size=10,
  meta_learning_rate=1e-4,
  meta_weight_decay=1e-4,
)

num_epochs = 30

model_names = []

num_runs = 1

fn = modal.Function.lookup("arc-prize-meta", "meta_train")
for i in range(num_runs):
  model_name = petname.generate(words=3, separator='_')
  fn_call = fn.spawn(model_name, num_epochs, model_type, model_params, train_params)
  print("Model name", model_name, model_type, fn_call.object_id)
  model_names.append(model_name)

print(model_names)

In [ ]:
import modal
from arc_prize.train import ARCTrainParams

num_epochs = 200

# model_names = ['subtly_moral_bee', 'newly_mint_kite']
# model_names = ["vastly_intent_frog_2"]
model_names = ["vastly_intent_frog_2"]

model_type = "vision_encoder"

# subtly_full_bull
train_params = ARCTrainParams(
  batch_size=8,
  learning_rate=1e-5,
  weight_decay=1e-5,
  dataset_dir=["/vol/data/chunked/html_dim_20_20241103", "/vol/data/chunked/re_arc_dim_20", "/vol/data/chunked/barc_1_dim_20", "/vol/data/chunked/barc_2_dim_20"],
  loss_class_weights={0: 0.2},
  train_steps_per_epoch=1000,
  eval_steps_per_epoch=100,
  warmup_epochs=10,
  refinement_ratio=0.25
)

# kindly_exact_beagle
# train_params = ARCTrainParams(
#   batch_size=16,
#   learning_rate=1e-5,
#   weight_decay=1e-5,
#   dataset_dir=["/vol/data/chunked/html_dim_12_20241108", "/vol/data/chunked/re_arc_dim_12", "/vol/data/chunked/barc_1_dim_12", "/vol/data/chunked/barc_2_dim_12"],
#   loss_class_weights={0: 0.2},
#   train_steps_per_epoch=1000,
#   eval_steps_per_epoch=200,
#   warmup_epochs=8,
#   refinement_ratio=0.25
# )

# train_params = ARCTrainParams(
#   batch_size=64,
#   learning_rate=5e-5,
#   weight_decay=1e-4,
#   dataset_dir=["/vol/data/chunked/html_dim_12_20241108", "/vol/data/chunked/re_arc_dim_12", "/vol/data/chunked/barc_1_dim_12", "/vol/data/chunked/barc_2_dim_12"],
#   loss_class_weights={0: 0.2},
#   train_steps_per_epoch=1000,
#   eval_steps_per_epoch=150,
#   warmup_epochs=8,
#   refinement_ratio=0.25
# )
# train_params = None

fn = modal.Function.lookup("arc-prize", "train_80gb")
for model_name in model_names:
  fn_call = fn.spawn(model_name, num_epochs, model_type, None, train_params)
  print("Model name", model_name, fn_call.object_id)
  


In [ ]:
import modal
from arc_prize.train import ARCModelState, EpochState
from arc_prize.vis import visualize_epochs

def visualize_group(model_names: list[str]):
  epochs = {}
  get_model = modal.Function.lookup("arc-prize", "get_model")
  for name in model_names:
    checkpoint_dict = get_model.remote(name)
    # checkpoint = ARCModelState(**get_model.remote(name))
    print(name, checkpoint_dict.get("model_type", None), checkpoint_dict["best_val_loss"], len(checkpoint_dict["epochs"]), checkpoint_dict["epochs"][-1], checkpoint_dict["model_params"])
    print(checkpoint_dict["train_params"])
    epochs[name] = checkpoint_dict["epochs"]


    # print(len(checkpoint.encoder_attn_weights))
    # for b, batch in enumerate(checkpoint.encoder_attn_weights):
    #   for i, layer in enumerate(batch):
    #     visualize_all_heads(layer, title=f"Batch {b}, layer {i}")
    

  visualize_epochs(epochs)

  


groups = [
  # ['kindly_huge_jennet', 'lovely_tidy_lab', 'solely_living_leech'], # BEST
  # ['weekly_enough_moose', 'gently_known_beagle', 'nicely_robust_rhino'], # 20x20 too slow
  # ['wildly_firm_husky', 'surely_brief_bug', 'fully_better_dodo'], # Amazing
  # ['wildly_steady_iguana', 'yearly_smart_donkey', 'mainly_polite_bison'], # Includes scale dataset
  # ['partly_vocal_piglet', 'neatly_needed_liger', 'firmly_game_weevil'], # Scale and diagonal
  # ['wholly_tops_heron', 'solely_eager_foal', 'deeply_one_skink'], # Tons of data
  # ['unduly_glad_swift', 'purely_steady_hornet', 'humbly_civil_donkey'], # Basic data
  # ["early_civil_beetle"],
  # ["really_fancy_kitten", "mildly_humble_tahr"],
  # ['solely_brief_shad', 'fairly_amazed_hyena', 'vastly_amazed_bobcat'],
  # ['barely_sound_viper', 'lively_key_goblin', 'wildly_fancy_glider'], # patch size 3
  # ['mildly_able_horse', 'vastly_normal_rhino', 'oddly_mint_clam'], # patch size 2
  # ['firmly_tops_adder', 'yearly_normal_puma', 'slowly_more_caiman'], # patch size 2, simpler embedding
  # ['safely_poetic_adder', 'vastly_close_horse', 'fairly_legal_insect', 'daily_actual_monkey'], # Patch embedding with ARC pos encoding
  # ['nicely_wired_mouse', 'freely_up_shrimp', 'hardly_loving_mullet', 'gladly_active_muskox']
  # ['mostly_normal_dog', 'lively_pure_hawk', 'rarely_tender_roughy'], # HUGE dataset re_arc_dim_12
  # ['daily_pro_cattle', 'newly_suited_finch', 'rarely_tender_roughy'], # 32 batch size, 64 dim
  # ['solely_sound_sponge', 'lively_sacred_egret'], # 32 batch size, 128 dim, 3+3 layers
  # ['nicely_pure_leech', 'namely_sure_emu'], # 128 dim, 4+4 layers
  # ['subtly_moral_bee', 'newly_mint_kite', 'solely_busy_skunk'], # Vision transformer large models
  # ['newly_mint_kite', 'namely_caring_mite', 'overly_enough_tomcat', 'subtly_known_panda', 'mildly_ruling_hog', 'simply_tight_fowl'],
  # ['subtly_known_panda', 'barely_clean_cicada', 'sadly_real_viper'], # 512 dim (12x12, 20x20, 30x30)
  # ['lively_fleet_goat'], # vision 20x20
  # ["kindly_living_spider", "gladly_prompt_koi"] # encoder vs normal
  # ["kindly_living_spider", "unduly_worthy_zebra", "daily_useful_squid", "vastly_intent_frog"],
  # ["kindly_exact_beagle", "jolly_picked_stud", "badly_moral_pika"],
  # ["kindly_exact_beagle", "jolly_picked_stud", "badly_moral_pika", "weekly_needed_buck", "lively_key_ape", "purely_ready_burro"]
  ["vastly_intent_frog_2", "kindly_exact_beagle", "jolly_picked_stud"]
]

# print([group for sublist in groups for group in sublist])
for group in groups:
  visualize_group(group)

In [33]:
import modal


eval_model = modal.Function.lookup("arc-eval", "evaluate_model")

output = eval_model.remote("vastly_intent_frog_2", ["/vol/data/eval_dim_12"], False, [0.2, 0.4])
# output = eval_model.remote("daily_useful_squid", ["/vol/data/re_arc_dim_12_small"], False, [0.8, 0.2], 100)
# output = eval_model.remote("overly_hip_egret", ["/vol/data/re_arc/00d62c1b.json"], True, 1)


In [9]:
import numpy as np

def score_output(output) -> tuple:
    correct = 0
    refined_correct = 0
    total = 0
    accuracy = 0.0
    refined_accuracy = 0
    close = 0
    for item in output:
        if np.array_equal(item["output_grid"], item["predictions"].squeeze(0)) is True:
            correct += 1
        if np.array_equal(item["output_grid"], item["refined_predictions"].squeeze(0)) is True:
            refined_correct += 1
        item_accuracy = (item["predictions"].squeeze(0) == item["output_grid"]).astype(float).mean()
        accuracy += item_accuracy
        if item_accuracy >= 0.95:
            close += 1
        refined_accuracy += (item["refined_predictions"].squeeze(0) == item["output_grid"]).astype(float).mean()
        total += 1

    refined_accuracy /= total
    accuracy /= total

    return (total, correct, accuracy, refined_correct, refined_accuracy, close)




In [ ]:
print(*score_output(output))


# print(f"Correct: {correct}/{total} (Accuracy: {accuracy:.4f}%)")
# print(f"Refined: {refined_correct}/{total} (Accuracy: {refined_accuracy:.4f}%)")
# print(f"Close: {close}/{total}")

In [ ]:
scores = []
temps = [0.0, 0.1, 0.4, 0.7, 0.9]
for first_temp in temps:
    for second_temp in temps:
        output = eval_model.remote("kindly_exact_beagle", ["/vol/data/eval_dim_12"], False, [first_temp, second_temp])
        print(first_temp, second_temp, *score_output(output))

In [41]:
import modal
fn = modal.Function.lookup("arc-eval", "finetune_and_predict")
output = fn.remote("jolly_picked_stud", ["/vol/data/eval_dim_12"], 15, None, [0.1, 0.2], 4, 1e-5, 1e-5, 0.995, 2)

In [ ]:
print(output[0])

In [81]:
import json

def convert_to_list(obj):
    if isinstance(obj, dict):
        return {k: convert_to_list(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_list(item) for item in obj]
    elif hasattr(obj, 'tolist'):  
        return obj.tolist()
    else:
        return obj

with open("experiments/jolly_picked_stud-eval_dim_12-finetune-2.json", "w") as file:
  json.dump(convert_to_list(output), file)

In [78]:
with open("experiments/jolly_picked_stud-eval_dim_12-finetune-1.json", "r") as file:
  experiment_dict = json.load(file)
  experiment = [{
    "task_id": item["task_id"],
    "grids": np.array(item["grids"]),
    "output_grid": np.array(item["output_grid"]),
    "predictions": np.array(item["predictions"]),
    "finetune_predictions": np.array(item["finetune_predictions"]),
    "refined_predictions": np.array(item["refined_predictions"])
  } for item in experiment_dict]


In [ ]:
import numpy as np


correct = 0
finetune_correct = 0
refined_correct = 0
total = 0
accuracy = 0.0
finetune_accuracy = 0
refined_accuracy = 0
close = 0
finetune_close = 0
refined_close = 0

correct_ids = []
incorrect_ids = []

trained_ids = ["03560426", "00dbd492", "0c786b71", "195ba7dc", "5ffb2104", "17b80ad2", "15696249", "12422b43", "4364c1c4", "0bb8deee", "0a1d4ef5", "0b17323b", "292dd178", "137f0df0", "0c9aba6e", "14754a24", "08573cc6", "17cae0c1", "12997ef3", "11e1fe23", "0f63c0b9", "1d398264", "351d6448", "0becf7df", "00576224", "332efdb3", "140c817e", "32e9702f", "0e671a1a", "12eac192"]


for item in output:
    if item["task_id"] not in trained_ids:
        continue
    if np.array_equal(item["output_grid"], item["predictions"][0]) is True:
        correct += 1
    if np.array_equal(item["output_grid"], item["finetune_predictions"][0]) is True:
        finetune_correct += 1
    if np.array_equal(item["output_grid"], item["refined_predictions"][0][0]) is True or np.array_equal(item["output_grid"], item["refined_predictions"][1][0]) is True:
        refined_correct += 1
    a = (item["predictions"][0] == item["output_grid"]).astype(float).mean()
    accuracy += a
    if a >= 0.95:
        close += 1
    f = (item["finetune_predictions"][0] == item["output_grid"]).astype(float).mean()
    finetune_accuracy += f
    if f >= 0.95:
        finetune_close += 1
    r = max((item["refined_predictions"][0][0] == item["output_grid"]).astype(float).mean(),
         (item["refined_predictions"][1][0] == item["output_grid"]).astype(float).mean())
    refined_accuracy += r
    if r >= 0.95:
        refined_close += 1
    total += 1

finetune_accuracy /= total
accuracy /= total
refined_accuracy /= total

print("Zero-shot Correct", correct, total, correct / total)
print("Zero-shot acc", accuracy)
print("Zero-shot close", close, total, close / total)

print("TTT Correct", finetune_correct, total, finetune_correct / total)
print("TTT acc", finetune_accuracy)
print("TTT close", finetune_close, total, finetune_close / total)

print("TTT + Refined Correct", refined_correct, total, refined_correct / total)
print("TTT + Refined acc", refined_accuracy)
print("TTT + Refined close", refined_close, total, refined_close / total)


In [ ]:
trained_ids = ["03560426", "00dbd492", "0c786b71", "195ba7dc", "5ffb2104", "17b80ad2", "15696249", "12422b43", "4364c1c4", "0bb8deee", "0a1d4ef5", "0b17323b", "292dd178", "137f0df0", "0c9aba6e", "14754a24", "08573cc6", "17cae0c1", "12997ef3", "11e1fe23", "0f63c0b9", "1d398264", "351d6448", "0becf7df", "00576224", "332efdb3", "140c817e", "32e9702f", "0e671a1a", "12eac192"]
task_ids = [item["task_id"] for item in experiment]
included = 0
for id in incorrect_ids:
  if trained_ids.__contains__(id):
    print(id)
    included += 1
len(correct_ids)
print(included)
print(task_ids)
overlap = list(set(trained_ids) & set(task_ids))
print(", ".join(task_ids))
len(task_ids)



In [ ]:
import torch
import matplotlib.pyplot as plt
from arc_prize.vis import visualize_tensors, visualize_all_heads
from arc_prize.vis import visualize_mean_mha_attention


def visualize_mean_attention(attention_weights: torch.Tensor, num_grids: int, grid_size: int):
    # Reshape the attention weights
    # From [4, 100, 900] to [4, 100, 9, 10, 10]
    num_heads = attention_weights.shape[0]
    reshaped_attention = attention_weights.view(
        num_heads, num_grids, grid_size, grid_size
    )

    # Calculate mean attention across the target sequence (dim=1)
    # mean_attention = reshaped_attention.mean(dim=1)  # Shape: [4, 9, 10, 10]
    mean_attention = reshaped_attention

    # Create a figure with subplots for each head
    fig, axes = plt.subplots(num_heads, num_grids, figsize=(20, 10))

    for head in range(num_heads):
        # Create a 3x3 grid of heatmaps
        for i in range(num_grids):
            grid_attention = mean_attention[head, i]

            # Add subplot within the head's subplot
            # sub_ax = ax.inset_axes([1/9])
            ax = axes[head, i]
            im = ax.imshow(grid_attention, cmap="viridis", interpolation="nearest")
            # ax.axis("off")

            if i == 0:
                ax.set_ylabel(f"Head {head + 1}", rotation=0, ha="right", va="center")

            # Add colorbar for each grid
            # plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

        # Remove ticks from the main subplot
        ax.set_xticks([])
        ax.set_yticks([])

    
    plt.tight_layout()
    plt.show()

for item in output:
#   if np.array_equal(item["output_grid"], item["predictions"]) is True:
    # visualize_tensors(torch.Tensor(item["grids"]).squeeze(0), torch.Tensor(item["output_grid"]).squeeze(0), torch.Tensor(item["predictions"]).squeeze(0).squeeze(0), torch.Tensor(item["refined_predictions"]).squeeze(0).squeeze(0))
    visualize_tensors(torch.Tensor(item["grids"]).squeeze(0), torch.Tensor(item["output_grid"]).squeeze(0), torch.Tensor(item["predictions"]).squeeze(0), torch.Tensor(item["refined_predictions"])[0][0].squeeze(0))
# print(torch.Tensor(item["decoder_sa_attn_weights"]).shape)
  # for i, layer in enumerate(torch.Tensor(item["decoder_mha_attn_weights"]).squeeze(0)):
  #   print(layer.shape)
  #   visualize_mean_attention(layer, 9, 30)
    

  # visualize_all_heads(layer, title=f"Layer {i}")
# for i, layer in enumerate(torch.Tensor(item["decoder_sa_attn_weights"]).squeeze(0)):
#     visualize_mean_sa_attention(layer)
    
